# Clasificación de los Simpsons!

> **IMPORTANTE**: Si este es el primer *notebook* que estudia, recomendamos arrancar con el *notebook* del *MNIST*, que contiene una complejidad inferior, el cual se puede encontrar [aca](./MNIST Classification.ipynb).

A continuación se detalla la implementación de un modelo que pueda identificar a los personajes de los simpsons en base a imagenes.

El *dataset* a usar se puede encontrar en el siguiente [link](https://www.kaggle.com/alexattia/the-simpsons-characters-dataset) de **Kaggle**. **Kaggle** es una página ampliamente útil donde se pueden encontrar *datasets* interesantes, participar en competencias, observar trabajos de otros usuarios, etc.

El *dataset* está conformado por imágenes de episodios. Si bien el *dataset* incluye informacion para poder efectuar **multiple object detection**, en este *notebook* nos vamos a dedicar a solucionar el problema de **imagen classification**. Esto quiere decir que, en cada imagen, aparece unicamente **un** personje, nuestro objetivo es clasificarlo. (A diferencia de **object detection**, donde la tarea sería detectar en una imagen recuadros de **todos** los personajes que aparecen en el mismo).

El modelo debería ser lo suficientemente potente como para detectar a los personajes incluso cuando no se encuentran en su representación más clara, como por ejemplo:

![img](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvs50EEmc52JZyQc7C992Lqod71vQZnzK9dDd7KuHeqTGG276Q)

Para este problema, vamos a implementar una CNN (Red Neuronal Convolucional) utilizando Tensorflow. Cada paso va a estar detallado y explicado.

## Requisitos

Para utilizar este *notebook*, se necesita descargar el dataset del siguiente [link](https://www.kaggle.com/alexattia/the-simpsons-characters-dataset/data). Recuerde la locación donde está la carpeta (descomprimida) del *dataset* pues se va a requerir luego. 

## Importando librerías

El siguiente comando importa las librerias requeridas por el resto del programa. Detallamos las más importantes:

* **numpy**: para el manejo en CPU de los tensores (vectores multidimensionales)
* **matplotlib**: para graficar en el notebook
* **tensorflow**: para construir y entrenar la red neuronal
* **utils**: modulo propio presente en `utils.py` con funciones auxiliares

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import glob
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
from skimage import io
from skimage import transform
import tensorflow as tf
import utils

%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow'

A continuación vamos a cargar los personajes que forman parte de este dataset. Para esto, cambie la siguiente variable global con la locación de su carpeta donde esta el *dataset*. Por ejemplo, `~/Data/simpsons-dataset`. Asegúrese de que la carpeta `simpsons_dataset` se encuentro de la carpeta que define en la variable `DATA_DIR`.

In [ ]:
# Place your directory here. For example:
DATA_DIR = "/Users/user/data/the-simpsons-characters-dataset/simpsons_dataset/"

Para cargar los personajes, ejecutar la siguiente función perteneciente al módulo `utils`. Este metodo va a cargar un mapa con los personajes. Se puede pasar un segundo argumento con la cantidad mínima de imágenes que necesita tener un personaje para ser considerado (cuantas más imagenes de personaje, más robusto el clasificador).

In [ ]:
map_characters = utils.load_characters(DATA_DIR)

Veamos los personajes que cargamos. Cada personaje esta identificado con un `id`:

In [ ]:
for k, v in map_characters.items():
    print("{} -> {}".format(k, v))

Ahora, vamos a cargar las imagenes a memoria. Utilizamos la siguiente funcion de `utils`:

In [ ]:
pics, labels = utils.load_pictures(DATA_DIR, map_characters)

`pics` es un `numpy array` que contiene las imagenes en el formato NHWC (Numero de imagenes x Altura x Ancho x Canales). 

`labels` es un `numpy array` con el *ground truth* con *One-hot encoding*, en el cual cada valor de verdad se representa como una distribucion de probabilidades por todas las posibles clases. Por ejemplo, para representar el *label* '3', el vector seria [0 0 0 1 0 0 0 0 ... 0 0 0] (osea, todos las clases en 0, menos la correspondiente, en 1).

Veamos las dimensiones de cada uno:

In [ ]:
print("Images shape:")
print(pics.shape)
print("Labels shape:")
print(labels.shape)

Veamos algunas imagenes del *dataset* con su respectivo *ground truth*. Para ver el codigo, ir a `utils.py`.

In [ ]:
utils.show_random_characters(pics, labels, map_characters)

### Como se representa el valor del pixel en las imagenes?

Como vimos antes, son imagenes en escala de grises (esto significa que tienen un solo canal). Ahora bien, veamos si los valores estan representados entre [0, 255] o entre [0, 1].

In [ ]:
print(np.unique(pics[0]))

Esto quiere decir que los valores no están normalizados! Más adelante vamos a tener que normalizarlos, es decir, tener valores entre 0 y 1. Esto es muy usual en Machine Learning, pues evita problemas numéricos y puede ayudar a la convergencia en la búsqueda de la solución.

## Separando en Entrenamiento / Validación

A continuación vamos a separar el *dataset* en 2:

1. *train*: van a ser las imagenes propias del entrenamiento.
2. *val*: estas imagenes no forman parte del entrenamiento, sino que sirven para medir la performance del modelo con *data* desconocida.

In [ ]:
pics_train, labels_train, pics_val, labels_val = utils.split(pics, labels, p=0.95)

Veamos las dimensiones de los nuevos conjuntos:

In [ ]:
print("Training data:")
print("X: {}".format(pics_train.shape))
print("Y: {}".format(labels_train.shape))
print()
print("Validation data:")
print("X: {}".format(pics_val.shape))
print("Y: {}".format(labels_val.shape))

## Definiendo el modelo

Definamos las siguientes variables:

In [ ]:
H, W, C = pics[0].shape
NUM_CLASSES = len(map_characters)

### 1) La arquitectura

La arquitectura, como se dijo antes, va a ser una CNN (Red Neuronal Convolucional). La misma va a consistir en una serie de convoluciones (con tecnicas de *pooling*, para reducir el dominio, para más información, [acá](http://ufldl.stanford.edu/wiki/index.php/Pooling)) acompañado de un clasificador de capas densas (*fully connected layers*).

La arquitectura se puede resumir en el siguiente esquema:

![img](http://adventuresinmachinelearning.com/wp-content/uploads/2017/04/Typical_cnn.png)

Estas topologías de redes (Convoluciones + Capas densas) son **estándar** para solucionar el problema de *Image Classification*. Para más información sobre CNN, pueden seguir el siguiente [tutorial](https://www.tensorflow.org/tutorials/layers).

La entrada de la red (el *input*) va a ser directamente la imagen. El *output* de la red va a ser el *one-hot encoding* conteniendo la clase de la imagen de entrada.

El flujo sería el siguiente:

1. Tomamos una imagen (que tiene un tamaño de HxW).
2. Se introduce en la red.
3. Se aplican una serie de convoluciones (filtros) acompañadas de *max pooling* para hacer *downsampling*.
4. Luego de los ultimos filtros de convolucion, se aplana el contenido (conocido como *activation map*).
5. Este vector plano entra a las capas densas de la red y fluye hacia la salida.
6. La capa de salida va a ser un vector de N elementos (siendo N la cantidad de personajes), donde cada uno representa un *score* de que esa imagen pertenezca a esa clase. Cuanto mayor sea el *score*, buscamos que sea más probable que la imagen pertenezca a esa clase (es decir, que sea ESE digito). Cuando la red esté entrenada, la clase (o el dígito) correcto va a ser aquel que tengo mayor *score*.

In [ ]:
def load_architecture():
    tf.reset_default_graph()
    
    x = tf.placeholder(tf.uint8, shape=[None, H, W, 3], name="x")
    y = tf.placeholder(tf.uint8, shape=[None, NUM_CLASSES], name="y")
    
    dropout_rate = tf.placeholder_with_default(0.3, shape=(), name="dropout_rate")
    
    is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
    
    init = tf.contrib.layers.xavier_initializer()
    
    out = tf.divide(x, 255)
    
    out = tf.layers.conv2d(out, filters=8, kernel_size=[3,3], activation=tf.nn.relu, kernel_initializer=init, padding="same")
    out = tf.layers.max_pooling2d(out, pool_size=(2, 2), strides=[2,2])
        
    out = tf.layers.conv2d(out, filters=16, kernel_size=[3,3], activation=tf.nn.relu, kernel_initializer=init, padding="same")
    out = tf.layers.max_pooling2d(out, pool_size=(2, 2), strides=[2,2])
        
    out = tf.layers.conv2d(out, filters=32, kernel_size=[3,3], activation=tf.nn.relu, kernel_initializer=init, padding="same")
    out = tf.layers.max_pooling2d(out, pool_size=(2, 2), strides=[2,2])
        
    out = tf.layers.conv2d(out, filters=64, kernel_size=[3,3], activation=tf.nn.relu, kernel_initializer=init, padding="same")
    out = tf.layers.max_pooling2d(out, pool_size=(2, 2), strides=[2,2])
        
    out = tf.contrib.layers.flatten(out)
    
    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)

    out = tf.layers.dense(out, units=512, activation=tf.nn.relu, kernel_initializer=init)
    
    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)

    out = tf.layers.dense(out, units=512, activation=tf.nn.relu, kernel_initializer=init)

    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)
    
    out = tf.layers.dense(out, units=512, activation=tf.nn.relu, kernel_initializer=init)
    
    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)
    
    out = tf.layers.dense(out, units=512, activation=tf.nn.relu, kernel_initializer=init)

    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)
    
    out = tf.layers.dense(out, units=NUM_CLASSES, kernel_initializer=init, name="out")
    
    return x, y, is_training, dropout_rate, out

### 2) La función de costo (*loss*)

La funcion de costo para este problema va a ser la entropía cruzada aplicada a la funcion softmax sobre la capa de salida. Expliquemos un poco esto:

En primer lugar, se computa la funcion softmax sobre la capa de salida (que son los *scores* de las clases). Esta funcion tiene la siguiente pinta:

![img](https://wikimedia.org/api/rest_v1/media/math/render/svg/e348290cf48ddbb6e9a6ef4e39363568b67c09d3)

Esta funcion mapea los *scores* a una distribucion de probabilidad, intensificando el valor del maximo (por ejemplo, si los scores hubieran sido [1.3, -0.2, 5.2], la funcion daria un vector ~[0.0197, 0.0044, 0.976]. Ahora, el *output* de la red está en terminos de probabilidad, al igual que el *ground truth*! (acuerdense que está en formato *One-hot encoding*).

Gracias a esto, definimos la entropia cruzada, que es una forma de relacionar dos distribuciones de probabilidad:

![img](https://wikimedia.org/api/rest_v1/media/math/render/svg/0cb6da032ab424eefdca0884cd4113fe578f4293)

En resumen, cuando la probabilidad de la clase correcta en el *output* sea relativamente baja, la entropia cruzada va a ser altisima. Cuando sea alta, la entropia va a ser baja. Vamos a intentar minimizar la *loss* (que es la entropia cruzada luego del softmax), para buscar este ultimo comportamiento.

In [ ]:
def load_loss(y, out):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out, name="mean_loss")
    loss = tf.reduce_mean(loss, name="loss")
    return loss

La *accuracy* mide el porcentaje de eficacia entre los *labels* y el *output* de la red.

In [ ]:
def load_accuracy(y, out):
    pred = tf.argmax(out, axis=-1)
    gt = tf.argmax(y, axis=-1)
    
    matches = tf.equal(pred, gt)
    
    return tf.reduce_mean(tf.cast(matches, tf.float32), name="acc")

### 3) La elección del minimizador

Vamos a estar utilizando *Adam*. Es una variante adaptativa a *Stochastic Gradient Descent* (SGD).

Para más información acerca de los minimizadores, leer el siguiente excelente blog [aqui](http://ruder.io/optimizing-gradient-descent/).

In [ ]:
def load_trainer(loss):
    opt = tf.train.AdamOptimizer()
    return opt.minimize(loss)

### Funciones complementarias

Las siguientes funciones son complementarias y no revisten de mayor importancia.

In [ ]:
def register_scalars(m):
    for k, v in m.items():
        tf.summary.scalar(k, v)

In [ ]:
def register_images(m):
    for k, v in m.items():
        tf.summary.image(k, v)

In [ ]:
def trainable_parameters():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    return total_parameters

### Modelo Final

La siguiente funcion junta todos los pasos anteriores para definir el modelo final. Esta funcion es la encargada de cargar el grafo en Tensorflow para luego correr la optimizacion.

La funcion retorna aquellos nodos del grafo necesarios para ser corridos luego.

In [ ]:
def load_model():
    x, y, is_training, dropout_rate, out = load_architecture()
    loss = load_loss(y, out)
    acc = load_accuracy(y, out)
    upd = load_trainer(loss)
    
    register_scalars({"info_loss": loss, "info_acc": acc})
    register_images({"input": x})

    info = tf.summary.merge_all()
    
    return x, y, is_training, dropout_rate, out, loss, acc, upd, info

## Entrenando el modelo

Tensorflow requiere:

1. Definir el grafo computacional (lo que hicimos antes)
2. Correr el grafo a traves de una `Session`.

A continuacion, definimos la sesión.

In [ ]:
def load_session():
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    return sess

Luego, definimos una funcion que encapsula todo el entrenamiento de la red, es decir, la optimizacion de la funcion de *loss* definida previamente.

Esta funcion recibe la sesion, el modelo, la data, la cantidad de epocas, el tamaño del *batch* y los *writers*, que sirven para hacer uso de la herramienta de visualizacion *tensorboard*.

In [ ]:
def train(sess, model, pics_train, labels_train, pics_val, labels_val, epochs, batch_size, train_writer, val_writer, use_dropout=False):
    N, _, _, _ = pics_train.shape
    idxs = np.arange(N)
    
    x, y, is_training, dropout_rate, out, loss, acc, upd, info = model
    
    d_rate = 0.4 if use_dropout else 0.
    
    i=0

    for ep in tqdm(range(epochs)):
        np.random.shuffle(idxs)
        pics_train = pics_train[idxs]
        labels_train = labels_train[idxs]

        for b in range(0, N, batch_size):
            X_batch = pics_train[b:b+batch_size]
            Y_batch = labels_train[b:b+batch_size]

            if X_batch.shape[0] < BATCH_SIZE:
                break

            graph_info, _ = sess.run([info, upd], feed_dict={x: X_batch, y: Y_batch, is_training: True, dropout_rate: d_rate})
            train_writer.add_summary(graph_info, i)
            
            graph_info, = sess.run([info], feed_dict={x: pics_val, y: labels_val, is_training: False, dropout_rate: d_rate})
            val_writer.add_summary(graph_info, i)
            
            i+=1

Por ultimo, definimos una funcion que nos va a permitir probar el modelo entrenado. Esta funcion simplemente ejecuta la red con las imagenes que se proveen como parametro. Retorna las inferencias (es decir, las clases "ganadoras") para cada imagen.

In [ ]:
def predict(imgs, model):
    x, y, is_training, dropout_rate, out, loss, acc, upd, info = model

    N, H, W, _ = imgs.shape
    fig=plt.figure(figsize=(20, 20))
    columns = 3
    rows = 3
    for i in range(1, columns*rows +1):
        idx = np.random.choice(range(N)) 
        img = imgs[idx]
        img_batch = np.reshape(img, [1, H, W, 3])
        graph_out, = sess.run([out], feed_dict={x: img_batch})
        char = np.argmax(np.squeeze(graph_out))
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        plt.title(map_characters[char])
    plt.show()

### *Overfitteando* data primero

Para ver que el modelo esta bien implementado y converge a una solución, primero tratemos que aprenda una pequeña porción del *dataset*. Esta técnica es muy usual para validar la implmenetación del modelo.

Obviamente, con pocos datos y un modelo potente, vamos a sufrir *overfitting*. Pero en este escenario no es un problema, pues solo estamos validando el modelo, y lo que queremos es ver si es capaz de llegar a una solucion. Es decir, estamos buscando el *overfitting*.

Si no esta familiarizado con el termino **overfitting**, recomendamos leer el siguiente [post](https://www.geeksforgeeks.org/underfitting-and-overfitting-in-machine-learning/).

Tomemos primero una pequeña porción del set de entrenamiento

In [ ]:
pics_train_sm, labels_train_sm = utils.get_small_dataset(pics_train, labels_train, p=0.01)

Analicemos su tamaño:

In [ ]:
print("New training dataset size: {}".format(pics_train_sm.shape[0]))

Veamos la cantidad de parametros *aprendibles* que tiene el modelo:

In [ ]:
model = load_model()
print("Trainable parameters: {}".format(trainable_parameters()))

Entrenemos el modelo ahora.

In [ ]:
EPOCHS = 20
BATCH_SIZE = 32
LOGS_DIR = "logs"

sess = load_session()

t_writer = tf.summary.FileWriter(os.path.join(LOGS_DIR, "overfit", "train"), graph=sess.graph)
v_writer = tf.summary.FileWriter(os.path.join(LOGS_DIR, "overfit", "val"), graph=sess.graph)

train(sess, model, pics_train_sm, labels_train_sm, pics_val, labels_val, EPOCHS, BATCH_SIZE, t_writer, v_writer)

En la carpeta `logs` van a poder tener informacion util para analizar el proceso de entrenamiento. Para verla, se necesita levantar `tensorboard`. Para esto, ir a la consola y ejecutar:

```
$ tensorboard --logdir ./logs
```

Se les va a abrir un *tab* en el navegador donde van a poder ver los graficos de entrenamiento en funcion de las épocas.

Veamos que es lo que aprendio (o memorizó en este caso), la red:

In [ ]:
predict(pics_train_sm, model)

Sin embargo, sabemos que estamos *overfitteando* porque el modelo no puede resolver imagenes que no ha visto antes. Decimos en este caso que la red no tiene poder de **generalización**.

In [ ]:
predict(pics_val, model)

### Entrenando con toda la data

Repetimos el proceso pero con todas las imagenes de entrenamiento

In [ ]:
model = load_model()
sess = load_session()

In [ ]:
EPOCHS = 70
BATCH_SIZE = 64
LOGS_DIR = "logs"

t_writer = tf.summary.FileWriter(os.path.join(LOGS_DIR, "all-1", "train"), graph=sess.graph)
v_writer = tf.summary.FileWriter(os.path.join(LOGS_DIR, "all-1", "val"), graph=sess.graph)

train(sess, model, pics_train, labels_train, pics_val, labels_val, EPOCHS, BATCH_SIZE, t_writer, v_writer, use_dropout=True)

Veamos visualmente algunos ejemplos de entrenamiento.

In [ ]:
predict(pics_train, model)

Veamos ahora si la red pudo generalizar. De ser así, debería tener una buena *performance* con imágenes que no ha visto duranete su proceso de entrenamiento.

In [ ]:
predict(pics_val, model)

Veamos a continuación cuáles son los ejemplos en el set de validación que la red no ha podido detectar correctamente.

Pueden sacar alguna conclusión viendo estos ejemplos?

In [ ]:
x, y, is_training, dropout_rate, out, loss, acc, upd, info = model

graph_out, = sess.run([out], feed_dict={x: pics_val})
pred = np.argmax(graph_out, axis=-1)
gt = np.argmax(labels_val, axis=-1)


mismatches = pics_val[pred != gt]

print(mismatches.shape)

predict(mismatches, model)

Veamos la performance *overall* que tuvo el modelo en la data de validacion.

In [ ]:
x, y, is_training, dropout_rate, out, loss, acc, upd, info = model

N, H, W, _ = pics_val.shape
graph_out, = sess.run([acc], feed_dict={x: pics_val, y: labels_val})
print("Overall accuracy: {0:.2f}%".format(100 * np.squeeze(graph_out)))

Recordemos que en este *notebook* no se hizo una busqueda de hiperparámetros para optimizar el modelo. Bajo este escenario, podemos decir que nuestro set de validacion es finalmente nuestro set de *test*. Para reportar la performance del modelo, podemos utilizar la *accuracy* de este set.

Si hubiésemos hecho una busqueda de hiperparámetros, tendríamos que haber dividir el *dataset* en *training, validation, testing*. Para más información de esto, ir [aquí](https://stats.stackexchange.com/questions/19048/what-is-the-difference-between-test-set-and-validation-set).